1. Kurulum

In [ ]:
!pip install tensorflow
!pip install gym
!pip install keras

In [ ]:
!pip install keras_rl2


In [ ]:
!pip install gym[all]

2. Env Test

In [1]:
import gym,random
env=gym.make("CartPole-v0")
states=env.observation_space.shape[0]
actions=env.action_space.n
episodes=10
for episode in range(1,episodes+1):
    state=env.reset()
    done=False
    score=0

    while not done:
        env.render()
        action=random.choice([0,1])
        n_state,reward,done,infor=env.step(action)
        score+=reward
    print(f"Episode:{episode}  Score:{score}")

c:\Users\mehmu\PycharmProjects\github repo\ReinforcementLGames\gymgames\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\mehmu\PycharmProjects\github repo\ReinforcementLGames\gymgames\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\mehmu\PycharmProjects\github repo\ReinforcementLGames\gymgames\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Episode:1  Score:24.0
Episode:2  Score:16.0
Episode:3  Score:35.0
Episode:4  Score:22.0
Episode:5  Score:38.0
Episode:6  Score:15.0
Episode:7  Score:14.0
Episode:8  Score:27.0
Episode:9  Score:59.0
Episode:10  Score:28.0


3. Model Oluşturma

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers import Adam

def build_model(states,actions):
    model=Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model
#del model
model=build_model(states,actions)
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


4. Agent with Keras-RL

In [8]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [9]:
def build_agent(model,actions):
    policy=BoltzmannQPolicy()
    memory=SequentialMemory(limit=50000,window_length=1)
    dqn=DQNAgent(model=model,memory=memory,policy=policy,
    nb_actions=actions,nb_steps_warmup=10,target_model_update=1e-2)
    return dqn

In [10]:
dqn=build_agent(model,actions)
dqn.compile(Adam(learning_rate=1e-3),metrics=['mae'])
dqn.fit(env,nb_steps=5000,visualize=False,verbose=1)

Training for 5000 steps ...
Interval 1 (0 steps performed)


c:\Users\mehmu\PycharmProjects\github repo\ReinforcementLGames\gymgames\lib\site-packages\keras\engine\training_v1.py:2078: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    1/10000 [..............................] - ETA: 2:40:38 - reward: 1.0000

c:\Users\mehmu\PycharmProjects\github repo\ReinforcementLGames\gymgames\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


 4993/10000 [=============>................] - ETA: 32s - reward: 1.0000done, took 33.307 seconds


5. Agent Reload 

In [11]:
dqn.save_weights('SavedWeights/5k/dqn_weights.h5f')

In [ ]:
dqn.load_weights('SavedWeights/5k/dqn_weights.h5f')

In [12]:
scores=dqn.test(env,nb_episodes=5,visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 5 episodes ...
Episode 1: reward: 144.000, steps: 144
Episode 2: reward: 128.000, steps: 128
Episode 3: reward: 140.000, steps: 140
Episode 4: reward: 137.000, steps: 137
Episode 5: reward: 136.000, steps: 136
137.0
